In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Aim**: Using train data, find the best model for splitting each annotation from case notes and for finding corresponding features from splitted-data.

**Datasets**

*Patient Note*: Text detailing important information related by the patient during the encounter.
*Feature*: A clinically relevant concept. A rubric describes the key concepts relevant to each case.
*Clinical Case (Train)*: The scenario the Standardized Patient presents to the test taker. Ten clinical cases are represented in this dataset.
*Test*: Selected instances to validate our model.
*Sample Submission*: A sample submission file in the correct format. Character spans indicating the location(s) of the feature within the note.

In [ ]:
import seaborn as sns
import string
import plotly.express as px
from wordcloud import WordCloud
import matplotlib.pyplot as plt
%matplotlib inline
from warnings import filterwarnings
filterwarnings("ignore")

In [ ]:
features = "/kaggle/input/nbme-score-clinical-patient-notes/features.csv"
sample_sub = "/kaggle/input/nbme-score-clinical-patient-notes/sample_submission.csv"
pat_notes = "/kaggle/input/nbme-score-clinical-patient-notes/patient_notes.csv"
test = "/kaggle/input/nbme-score-clinical-patient-notes/test.csv"
train = "/kaggle/input/nbme-score-clinical-patient-notes/train.csv"

pat_notes = pd.read_csv(pat_notes)
features = pd.read_csv(features)
sample_sub = pd.read_csv(sample_sub)
train = pd.read_csv(train)
test = pd.read_csv(test)

In [ ]:
pat_notes

In [ ]:
pat_notes.nunique()

In [ ]:
pat_notes["pn_history"].iloc[16].split(".")

In [ ]:
features

In [ ]:
features.nunique()

In [ ]:
train

In [ ]:
pat_notes["pn_history"][16][696:724]

# notes per case 

In [ ]:
fig = px.histogram(pat_notes["case_num"])
fig.update_layout(bargap = 0.2)

# feature per case (train)

In [ ]:
fig = px.histogram(train["case_num"])
fig.update_layout(bargap = 0.2)

In [ ]:
pat_notes["pn_history"][0]

# empty annotations

In [ ]:
train["annotation"] = train["annotation"].apply(eval)
train["location"] = train["location"].apply(eval)
train["annot_count"] = 0
for i in range(len(train)):
    train["annot_count"][i] = len(train["annotation"][i])
fig = px.histogram(train["annot_count"])
fig.update_layout(bargap = 0.2)

**NOTE**
As seen there is not much train data is exist after 4th case. So, there is no need to model for every case and it is important to develop a general model to train with using 0-3th cases.

In [ ]:
text = " ".join(list(features["feature_text"]))
wcloud = WordCloud().generate(text)
plt.imshow(wcloud)
plt.axis("off")
plt.show()

In [ ]:
for case in features["case_num"].unique():
    text = " ".join(list(features[features["case_num"]==case]["feature_text"]))
    wcloud = WordCloud().generate(text)
    plt.imshow(wcloud)
    plt.axis("off")
    plt.title("case {}".format(case))
    plt.show()

In [ ]:
sample_sub

In [ ]:
train_merge = train.merge(pat_notes, on=["pn_num", "case_num"], how ="left")
train_merge = train_merge.merge(features, on = ["feature_num", "case_num"], how ="left")
train_merge.head(3)